In [1]:
import transformers
import os
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
## VERSION CHECK
print(transformers.__version__)
print(torch.__version__)

### Data Exploration & Preparation

1. bert-base-uncased
- 12-layer, 768-hidden, 12-heads, 110M parameters.
- Trained on lower-cased English text.

2. bert-large-uncased
- 24-layer, 1024-hidden, 16-heads, 340M parameters.
- Trained on lower-cased English text.

3. bert-base-cased
- 12-layer, 768-hidden, 12-heads, 110M parameters.
- Trained on cased English text.

4. bert-large-cased
- 24-layer, 1024-hidden, 16-heads, 340M parameters.
- Trained on cased English text.

#### major error issue
 
- error issue: dropout(): argument 'input' (position 1) must be Tensor, not str when using Bert with Huggingface
- transformers version issue: transformers 3.0에서 transformers 4.x버전으로 옮기면서 나타나는 문제.
    
- 해법: 모델 함수에 return_dict=False
    
reference: https://huggingface.co/docs/transformers/migration


In [3]:
# 다른 pretrained model을 사용하고 싶다면 위에서 원하는 이름을 선택해서 아래 변수명을 변경한다.
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME, return_dict=False) 
BATCH_SIZE = 8
tag = 'sentiment'

Downloading: 100%|██████████| 29.0/29.0 [00:00<00:00, 14.6kB/s]
Downloading: 100%|██████████| 208k/208k [00:00<00:00, 511kB/s] 
Downloading: 100%|██████████| 426k/426k [00:00<00:00, 559kB/s]  


In [4]:
def load_dataset(tag):
    """
    Description: 본 데이터 셋에서는 응답자가 의미없다는 답변을 할 수 있다. 
    의미 없다는 답변을 받은 데이터는 'irrelevant', 의미있는 답변이 담긴 데이터 프레임을 받은 데이터 프레임은 'sentiment'로 선택한다.
    ---------
    Arguments
    ---------
    tag : str
        'irrelevant', 'sentiment'로 원하는 데이터를 뽑아낸다.
    ---------    
    Return: pandas.Dataframe
            
    ---------
    
    """
    if tag == 'irrelevant':
        train_df = pd.read_csv('data/irrelevant_train.tsv', sep='\t')
        valid_df = pd.read_csv('data/irrelevant_test.tsv', sep='\t')
    elif tag == 'sentiment':
        train_df = pd.read_csv('data/sentiment_train.tsv', sep='\t')
        valid_df = pd.read_csv('data/sentiment_test.tsv', sep='\t')
    else:
        train_df = None
        valid_df = None
    
    return train_df, valid_df

In [5]:
class SentimentDataset(Dataset):
    """
    Description: 불러온 데이터프레임에서 필요한 정보를 뽑아내고, 임베딩한다.
    ---------
    Arguments
    ---------
    sentences: str
        문장에 대한 정보를 저장
    labels : int
        들어온 문장의 id를 가지고 있는다.
    tokenizer: str
        어떤 tokenizer를 사용할지 지정한다.
    max_len: int
        문장의 최대 길이를 지정하여 지나치게 긴 문장을 잘라낸다. max_len이 길어지면, padding을 해야하기에 연산이 느려진다.
    ---------    
    Return: dict
        dict안의 value와 item은 다음과 같다.
    ---------
    sentence: str
        input에 들어갈 문장정보
    input_ids: tensor
        tokenizer가 임베딩한 문장의 정보
    attention_mask: tensor
        최대 길이로 지정한 512에서 단어가 차지하는 길이에 대한 정보        
    labels: int
        분류 모델이 분류할 타겟 변수
    """
    def __init__(self, sentences, labels, tokenizer, max_len):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, item):
        sentence = str(self.sentences[item])
        label = self.labels[item]
        
        ## torch가 인식 할 수 있도록 문장을 embedding한다.
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        return {
            'sentence': sentence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [6]:
train_df, valid_df = load_dataset(tag)
df_train, df_test = train_test_split(train_df, test_size=0.2, random_state=1234)
print(df_train.shape, df_test.shape)

(32887, 4) (8222, 4)


In [7]:
train_dataset = SentimentDataset(df_train.sentence.values, df_train.sentiment.values, tokenizer, max_len=512)
## gpu가 감당 할 수 있게, small batch를 사용한다.
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=0)

test_dataset = SentimentDataset(df_test.sentence.values, df_test.sentiment.values, tokenizer, max_len=512)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=0)

In [8]:
## data loader를 사용한다.
data = next(iter(train_dataloader))
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['labels'].shape)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


torch.Size([8, 512])
torch.Size([8, 512])
torch.Size([8])


### Training

In [15]:
class SentimentClassifier(nn.Module):
    """
    Description: torch의 nn.Module을 사용해서 분류기 클래스를 만든다.
    ---------
    """
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
        
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        output = self.drop(pooled_output)
        return self.out(output)

In [16]:
## bert모델 를 정의한다. class는 2개.
model = SentimentClassifier(2)
model = model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
EPOCHS = 50 ## 바꿀 수 있는 parameter
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_dataloader) * EPOCHS
## lr를 줄이기 위해서 scheduler를 사용한다.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)
## loss function을 정의한다.
loss_fn = nn.CrossEntropyLoss().to(device)

In [27]:
scheduler

In [ ]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    """
    Description: train을 해주는 모듈
    ---------
    Arguments
    ---------
    model: nn.module
        정의한 모델
    loss_fn : CrossEntropyLoss()
        손실함수
    tokenizer: PreTrainedTokenizer
        앞에서 지정한 tokenizer
    max_len: int
        지정한 문장의 최대길이
    optimizer: AdamW 
        사용하고자 하는 optimizer
    device: device(type='cuda')
        gpu사용 혹은 cpu사용
    scheduler: scheduler
        사용할 scheduler
    n_examples : int
        전체 분류기에 사용할 자료의 수
    ---------    
    Return: train_accuracy, train_loss
    ---------
    """
    ## model을 train mode로 변환.
    model = model.train()
    losses = []
    correct_predictions = 0
    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["labels"].to(device)
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)
        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())
        loss.backward()
        ## gradient normalization
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
    """
    Description: train된 모델을 evaluation을 해주는 모듈
    ---------
    Arguments
    ---------
    model: nn.module
        정의한 모델
    loss_fn : CrossEntropyLoss()
        손실함수
    device: device(type='cuda')
        gpu사용 혹은 cpu사용
    n_examples : int
        전체 분류기에 사용할 자료의 수
    ---------    
    Return: eval_accuracy, eval_loss
    ---------
    """
    ## model을 eval mode로 변환.
    model = model.eval()
    losses = []
    correct_predictions = 0
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["labels"].to(device)
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)
            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())
    return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
history = defaultdict(list)
best_accuracy = 0
for epoch in range(EPOCHS):

    train_acc, train_loss = train_epoch(
        model,
        train_dataloader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(df_train)
    )
    
    val_acc, val_loss = eval_model(
        model,
        test_dataloader,
        loss_fn,
        device,
        len(df_test)
    )
    
    print(f'Epoch [{epoch + 1}/{EPOCHS}] Train loss: {train_loss} acc: {train_acc} | Val loss: {val_loss} acc: {val_acc}')

    print()
    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)
    if val_acc > best_accuracy:
        model_dir = 'model/'+tag+'_BERT_model.bin'
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        torch.jit.save(torch.jit.script(model), model_dir)
        best_accuracy = val_acc

### Evaluation

In [ ]:
valid_dataset = SentimentDataset(valid_df.sentence.values, valid_df.sentiment.values, tokenizer, max_len=512)
valid_dataloader = DataLoader(valid_dataset, batch_size=1, num_workers=0)

In [ ]:
result = eval_model(model, valid_dataloader, loss_fn, device, len(valid_dataset))
print(result)

### Load model and Get probabilities

In [ ]:
model = SentimentClassifier(2)
model.load_state_dict(torch.load('model/mobilebert_model_67.pt', map_location='cpu'))
## 안되면 model.load_state_dict(torch.load('model/mobilebert_model_67.pt', map_location='cpu'), strict = False)
model = model.to('cpu')

In [ ]:
def inference(input_text, model):
    """
    Description: 특정 문장이 들어오면 input text를 embedding하고, inference 해주는 모듈
        api serving에서 사용.
    ---------
    Arguments
    ---------
    input_text: str
        사용자가 넣을 문장 정보.
    model: model
        사용자가 지정한 모델 정보.
    ---------    
    Return: 0과 1 사이의 결과 값
    ---------
    """
    encoded_review = tokenizer.encode_plus(
        review_text,
        max_length=512,
        add_special_tokens=True,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    
    logits = model(input_ids, attention_mask)
    softmax_prob = torch.nn.functional.softmax(logits, dim=1)
    _, prediction = torch.max(softmax_prob, dim=1)
    
    return softmax_prob, prediction
        

In [ ]:
# Example code
review_text = "any line of news to test code"
class_prob, pred = inference(review_text, model)
print(class_prob.detach().cpu().numpy()[0])
print(pred.detach().cpu().numpy()[0])

In [ ]:
from tqdm import tqdm
def save_results(df, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for idx, row in tqdm(df.iterrows()):
            news_id = row['id']
            text = row['sentence'].replace('\t',' ')
            sentiment = row['sentiment']
            class_prob, pred = inference(text, model)

            class_prob = [str(x) for x in class_prob.detach().cpu().numpy()[0]]
            pred = pred.detach().cpu().numpy()[0]

            result = str(news_id).replace('\t','')+'\t'+text+'\t'+'\t'.join(class_prob)+'\t'+str(pred)+'\t'+str(int(sentiment)).replace('\t','')
            
            f.write(result+'\n')

In [ ]:
save_results(train_df, 'data/'+tag+'_bert_prediction_train.csv')

In [ ]:
save_results(valid_df, 'data/'+tag+'_bert_prediction_test.csv')

## 보류: APEX

### amp Mixed Precision

float16으로 Type Casting 되는 것이 빠른 연산(Linear Layer, Conv Layer etc.)은 float16으로 변환해서 연산을 수행하는 것이 가능. 
float16으로 변환시키는 경우 autocast와 GradScaler를 사용한다. 
- autocast: autocast는 with 문과 함께 선언해서 사용하면, with내부의 토치 연산들은 mixed precision로 실행된다. model의 forward 연산과, loss 계산 연산을 with문 아래에 위치 시켜야 한다.
- GradScaler: float16으로 연산하면, foward-pass 이후의 backward-pass에서 그라디언트가 너무 작아져서 underflow가 일어나기도 한다. GradScaler는 이를 방지하기 위해 backward시 float32로 실행한다.

- amp은 float16연산 이외에도 DataParallel 기능을 지원한다. 특히 여러 gpu를 동시에 사용할때 DistributedDataParallel을 통해 하나의 작업에 여러 gpu를 사용할 수 있다. 

#### 결과 기록
- APEX를 사용했을때, 제대로 학습이 되지 않고 전부 1로 LABALING한다.
- 이 원인에 대해서 찾아볼 것.

#### reference
- amp module에 대한 torch 공식 튜토리얼: https://pytorch.org/docs/stable/amp.html
- amp module의 각 method에 대한 설명: https://pytorch.org/docs/stable/notes/amp_examples.html
- amp에 대한 한글 번역본: https://runebook.dev/ko/docs/pytorch/amp

In [ ]:
EPOCHS = 50 ## 바꿔야할 파라미터

use_amp = True
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_dataloader) * EPOCHS
## GradScaler를 넣는 부분.
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=3,
    num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0
    for d in data_loader:
        ## autocast를 넣어주는 부분.
        with torch.cuda.amp.autocast(enabled=use_amp):
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["labels"].to(device)
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)
            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())
        scaler.scale(loss).backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    return correct_predictions.double() / n_examples, np.mean(losses)

###  DYNAMIC QUANTIZATION

- 참고자료  
    - BERT 모델 동적 양자화하기: https://tutorials.pytorch.kr/intermediate/dynamic_quantization_bert_tutorial.html

- 의문사항
    - AMP와 qunatization을 동시에 사용하면 안되나? 
    - amp에 대한 공식 설명: https://pytorch.org/docs/stable/amp.html
    - amp의 기능에 대한 설명: https://aimaster.tistory.com/83
    - https://runebook.dev/ko/docs/pytorch/amp
- 성능이 원하는 만큼 나오지 않는다.

In [ ]:
# DYNAMIC QUANTIZATION
model = SentimentClassifier(2)
model.load_state_dict(torch.load('model/mobilebert_model_67.pt', map_location='cpu'))
model = model.to('cpu')

In [ ]:
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

### STATIC QUANTIZATION